In [45]:
# !pip install feather-format
import pyarrow.feather as feather
titanic_train = feather.read_feather('titanic_train.feather')
titanic_train.head()
titanic_test = feather.read_feather('titanic_test.feather')
titanic_test.head()

,grupo,id,HomePlanet,CryoSleep,deck,room_num,side,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,fname,lname,gastos
0,13.0,1.0,Earth,True,G,3.0,S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly,Carsoning,0.0
1,18.0,1.0,Earth,False,F,4.0,S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome,Peckers,2832.0
2,19.0,1.0,Europa,True,C,0.0,S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih,Unhearfus,0.0
3,21.0,1.0,Europa,False,C,1.0,S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz,Caltilter,7418.0
4,23.0,1.0,Earth,False,F,5.0,S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence,Harperez,645.0


In [108]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score 
import pandas as pd

aplicaremos los siguientes modelos
* TSNE
* K-means
* Linear Regression
* Logistic Regression 
* Random Forest
* Boosted Forest
* PCA
* KNN classifier
* Support Vector Machine

Para optimizar los hiperparametros utilizaremos grid search

Manual de supervivencia
1. Escoje un modelo
1. Escoje los hiper-parámetros del modelo
1. Prepara el dataset en features matrix y el vector target
1. Divide el dataset en train y test
1. Aplica el algoritmo a tus datos de entrenamiento
1. Aplica el algoritmo a tus datos de prueba
1. Evalúa el modelo
1. Aplica el modelo en nuevos datos

In [110]:
# preparamos nuestro dataset en featura matrix y vector target
y_data = titanic_train["Transported"]
X_data = titanic_train.drop("Transported", axis=1)

In [119]:
# dividimos en train y test
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, random_state=0)
# creamos dummy variables 
dummies_deck=pd.get_dummies(X_train['deck'],drop_first=True)
dummies_destination=pd.get_dummies(X_train['Destination'],drop_first=True)
dummies_home=pd.get_dummies(X_train['HomePlanet'],drop_first=True)
X_train_lr = X_train.drop(["lname", "fname", "Destination", "CryoSleep", "HomePlanet", "deck", "side"],axis=1)
X_train_lr=pd.DataFrame(X_train_lr)
X_train_lr=pd.concat([X_train_lr,dummies_deck ,dummies_destination,dummies_home], axis=1)
X_train=X_train_lr

In [116]:
# regresion logistica
classifier=LogisticRegression()
hyper_param_grid = {'penalty':['none','l1','l2','elasticnet'],
                    'dual':[True,False],
                    'C':[0.001,0.01,0.1,1,1.5],
                    'fit_intercept':[True,False],
                    'intercept_scaling':[0.001,0.01,0.1,1,1.5],
                    'solver':['newton-cg','lbfgs','liblinear','sag','saga']
                   },

grid_search = GridSearchCV(classifier, hyper_param_grid, cv = 5, verbose = 3)

grid_search.fit(X_train_lr, y_train)

Fitting 5 folds for each of 2000 candidates, totalling 10000 fits
[CV 1/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=0.001, penalty=none, solver=newton-cg;, score=nan total time=   0.0s
[CV 2/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=0.001, penalty=none, solver=newton-cg;, score=nan total time=   0.0s
[CV 3/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=0.001, penalty=none, solver=newton-cg;, score=nan total time=   0.0s
[CV 4/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=0.001, penalty=none, solver=newton-cg;, score=nan total time=   0.0s
[CV 5/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=0.001, penalty=none, solver=newton-cg;, score=nan total time=   0.0s
[CV 1/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=0.001, penalty=none, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=0.001, penalty=none, solver

C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 446, in _check_solver
    raise ValueError("Solver %s supports only "
ValueError: Solver newton-cg supports only dual=False, got dual=True

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Es

[CV 5/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=0.001, penalty=none, solver=sag;, score=nan total time=   0.0s
[CV 1/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=0.001, penalty=none, solver=saga;, score=nan total time=   0.0s
[CV 2/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=0.001, penalty=none, solver=saga;, score=nan total time=   0.0s
[CV 3/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=0.001, penalty=none, solver=saga;, score=nan total time=   0.0s
[CV 4/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=0.001, penalty=none, solver=saga;, score=nan total time=   0.0s
[CV 5/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=0.001, penalty=none, solver=saga;, score=nan total time=   0.0s
[CV 1/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=0.001, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 2/5] END C=0.001, dual=True, fit_intercept=True, 

C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 446, in _check_solver
    raise ValueError("Solver %s supports only "
ValueError: Solver saga supports only dual=False, got dual=True

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimat

C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1358, in fit
    self.coef_, self.intercept_, n_iter_ = _fit_liblinear(
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 974, in _fit_liblinear
    solver_type = _get_liblinear_solver_type(multi_class, penalty, loss, dual)
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 830, in _get_liblinear_solver_type
    raise ValueError('Unsupported set of arguments: %s, '
ValueError: Unsupported set of arguments: The combination of 

...

C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 446, in _check_solver
    raise ValueError("Solver %s supports only "
ValueError: Solver newton-cg supports only dual=False, got dual=True

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Es

[CV 4/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=1, penalty=none, solver=newton-cg;, score=nan total time=   0.0s
[CV 5/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=1, penalty=none, solver=newton-cg;, score=nan total time=   0.0s
[CV 1/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=1, penalty=none, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=1, penalty=none, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=1, penalty=none, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=1, penalty=none, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=1, penalty=none, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=0.001, dual=True, fit_intercept=True, intercept_scali

C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 446, in _check_solver
    raise ValueError("Solver %s supports only "
ValueError: Solver saga supports only dual=False, got dual=True

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimat

[CV 5/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=1, penalty=none, solver=saga;, score=nan total time=   0.0s
[CV 1/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=1, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 2/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=1, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 3/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=1, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 4/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=1, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 5/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=1, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 1/5] END C=0.001, dual=True, fit_intercept=True, intercept_scaling=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END C=0.001, dual=True, fit_intercept=True, intercept_scalin

KeyboardInterrupt: 

In [ ]:
params, scores = [], [] 
params.append(grid.best_params_)
scores.append(grid.best_score_)

In [115]:
# regresion lineal
modelo = LinearRegression(fit_intercept = True)
modelo.fit(X_train, y_train) 

C:\Users\javie\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


ValueError: Unable to convert array of bytes/strings into decimal numbers with dtype='numeric'

In [77]:
print("Coeficientes: {}".format(modelo.coef_))

print("Interceptor: {}".format(modelo.intercept_))

Coeficientes: [ 5.07739384e-06  3.08253532e-02 -4.52686556e-02 -2.47787687e-04
  5.40763800e-03 -1.31880233e-04  8.10764757e-05  5.27426820e-05
 -5.83890759e-05 -5.34261317e-05 -4.05683266e-05]
Interceptor: 0.7130937235178567


In [78]:
## 6. Aplica el algoritmo a tus datos de prueba
y_from_model = modelo.predict(X_test)
## 7. Evalúa el modelo
modelo.score(X_test, y_test) ## Devuelve el R^2

0.18922539408346184

In [102]:
params.append(grid.best_params_)
scores.append(grid.best_score_)

{'criterion': 'entropy', 'n_estimators': 1000}
0.8999846978892503


In [101]:
# random forest
parameters = {'n_estimators':[1,10,100, 1000], 'criterion': ['gini', 'entropy', 'log_loss']}
model = RandomForestClassifier()
model.fit(X_train, y_train)
grid = GridSearchCV(model, parameters)
grid.fit(X_train, y_train)

y_model = model.predict(X_test)
## Accuracy
print("Accuracy: {}".format(accuracy_score(y_test, y_model)))
## Métricas clasicas de Clasificadores
print(metrics.classification_report(y_model, y_test))

C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\javie\anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\javie\anaconda3\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\javie\anaconda3\lib\site-packages\joblib\parallel.py", line 779, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\javie\anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\javie\anaconda3\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\javie\anaconda3\lib\site-packages\joblib\parallel.py", line 779, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\javie\anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\javie\anaconda3\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\javie\anaconda3\lib\site-packages\joblib\parallel.py", line 779, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\javie\anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\javie\anaconda3\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\javie\anaconda3\lib\site-packages\joblib\parallel.py", line 779, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

Accuracy: 0.9020239190432383
              precision    recall  f1-score   support

       False       0.89      0.91      0.90      1054
        True       0.91      0.89      0.90      1120

    accuracy                           0.90      2174
   macro avg       0.90      0.90      0.90      2174
weighted avg       0.90      0.90      0.90      2174



In [102]:
params.append(grid.best_params_)
scores.append(grid.best_score_)

{'criterion': 'entropy', 'n_estimators': 1000}
0.8999846978892503


In [100]:
# boosted forest
parameters = {'n_estimators':[1,10,100], 'loss': ['deviance', 'exponential', 'log_loss'], 'learning_rate':[0.1,1,5]}
model = GradientBoostingClassifier()
model.fit(X_train, y_train)
grid = GridSearchCV(model, parameters)
grid.fit(X_train, y_train)

y_model = model.predict(X_test)
## Accuracy
print("Accuracy: {}".format(accuracy_score(y_test, y_model)))
## Métricas clasicas de Clasificadores
print(metrics.classification_report(y_model, y_test))

C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 448, in fit
    self._check_params()
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 239, in _check_params
    raise ValueError("Loss '{0:s}' not supported. ".format(self.loss))
ValueError: Loss 'log_loss' not supported. 

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for 

C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 448, in fit
    self._check_params()
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 239, in _check_params
    raise ValueError("Loss '{0:s}' not supported. ".format(self.loss))
ValueError: Loss 'log_loss' not supported. 

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for 

C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 448, in fit
    self._check_params()
  File "C:\Users\javie\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 239, in _check_params
    raise ValueError("Loss '{0:s}' not supported. ".format(self.loss))
ValueError: Loss 'log_loss' not supported. 

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\javie\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for 

Accuracy: 0.9038638454461821
              precision    recall  f1-score   support

       False       0.88      0.92      0.90      1038
        True       0.92      0.89      0.91      1136

    accuracy                           0.90      2174
   macro avg       0.90      0.90      0.90      2174
weighted avg       0.90      0.90      0.90      2174



In [104]:
params.append(grid.best_params_)
scores.append(grid.best_score_)

In [ ]:
# Support vector machine 'C':[0.5, 1, 2],
parameters = {'kernel':['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'], 'gamma':['scale', 'auto'] }

model = SVC()
model.fit(X_train, y_train)
grid = GridSearchCV(model, parameters)
grid.fit(X_train, y_train)

y_model = model.predict(X_test)
## Accuracy
print("Accuracy: {}".format(accuracy_score(y_test, y_model)))
## Métricas clasicas de Clasificadores
print(metrics.classification_report(y_model, y_test))

In [103]:
params.append(grid.best_params_)
scores.append(grid.best_score_)

{'criterion': 'entropy', 'n_estimators': 1000}
0.8999846978892503


In [ ]:
# KNN
classifier=KNeighborsClassifier()
hyper_param_grid = {'n_neighbors':[2,5,10,12,15],
                    'weights':['uniform','distance'],
                    'algorithm':['auto','ball_tree','kd_tree'],
                    'leaf_size':[20,30,40],
                    'p':[1,2]
                   },

grid_search = GridSearchCV(classifier, hyper_param_grid, cv = 5, verbose = 3)

grid_search.fit(X_train, y_train)

In [103]:
params.append(grid.best_params_)
scores.append(grid.best_score_)

{'criterion': 'entropy', 'n_estimators': 1000}
0.8999846978892503


In [ ]:
for model, param, score in zip (names, params, scores)
    print(model + param + score)

In [3]:
def test_roc(model, data, labels):
    if hasattr(model, "decision_function"):
        predictions = model.decision_function(data)
    else:
        predictions = model.predict_proba(data)[:,1]
    fpr, tpr, thr = sklearn.metrics.roc_curve(labels, predictions)
    return fpr, tpr, thr

def test_conf(model, data, labels, thr):
    if hasattr(model, "decision_function"):
        predictions = model.decision_function(data)
    else:
        predictions = model.predict_proba(data)[:,1]
    mat = confusion_matrix(labels, predictions>=thr)
    return mat

In [5]:
for name, model in zip(classifier_names, classifiers):
    fpr, tpr, thr = test_roc(model, test_data, test_labels)
    plt.plot(fpr, tpr, label=name)
    print(name+' AUC:{}'.format(sklearn.metrics.auc(fpr, tpr)))
    J = tpr - fpr
    ix = argmax(J)
    best_thresh = thr[ix]
    print(name+' Best Threshold=%f' % (best_thresh))
    mat = test_conf(model, test_data, test_labels, best_thresh)
    print(mat)
    print()

NameError: name 'classifiers' is not defined